In [2]:
from unet import UNet

model = UNet(n_channels=8, n_classes=1, bilinear=True)

In [3]:
import torch
from torch.utils.data import Dataset
import tifffile
import os

class TiffSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = sorted(os.listdir(image_dir))
        self.masks = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])
        image = tifffile.imread(img_path)  # shape: (H, W, 8)
        mask = tifffile.imread(mask_path)  # shape: (H, W) oder (H, W, 1)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            image, mask = self.transform(image, mask)
        return image, mask

In [4]:
from torch.utils.data import DataLoader

train_dataset = TiffSegmentationDataset('C:\\Users\\licht\\Downloads\\Brick_Data_Train\\Brick_Data_Train\\Image', 'C:\\Users\\licht\\Downloads\\Brick_Data_Train\\Brick_Data_Train\\Mask')
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [5]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

In [9]:
# Alle Gewichte einfrieren
for param in model.parameters():
    param.requires_grad = False

# Nur die Parameter der letzten Ebene(n) trainierbar machen
for param in model.outc.parameters():  # 'outc' ist meist der finale Conv-Layer im UNet
    param.requires_grad = True

In [10]:
for epoch in range(10):
    model.train()
    for images, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 